In [ ]:
import pickle 
Stream_req = [pickle.load(open('/home/deng/bbserverlees/Azure/inter_data/g_app_1_y_brr.pkl', 'rb'))]



In [ ]:
Stream_req

In [ ]:
action_functions = ['lstm', 'mnist', 'resnet18', 'resnet34','resnet50', 'mobilenet4', 'shufflenet']
action_functions = ['lstm-cloud', 'mnist-cloud', 'resnet18-cloud', 'resnet34-cloud','resnet50-cloud', 'mobilenet-cloud', 'shufflenet-cloud']


base_url = 'http://127.0.0.1:8001/api/v1/namespaces/kl/services/{function}:8080/proxy'
res_col = ["function", "invoke_time", "timeModelLoad", "startTime", "endTime", "excutionTime","response_time", "run_time","ip"]
cwd = os.getcwd()

In [ ]:
import aiohttp
import asyncio
import nest_asyncio
import time
import pandas as pd
import requests
import json
import os

nest_asyncio.apply()

openfaas_base_url = 'http://serverless.siat.ac.cn:31112/function/{function_name}.openfaas-fn'


async def invoke_function(req, session,fname):
    url = openfaas_base_url.format(function_name=fname)
    res = requests.get(url)
    invoke_time = time.time()
    url = openfaas_base_url.format(function_name=fname)
    # res = requests.get(url)
    res = await session.request('GET', url)
    if res.status_code == 200:
        text = res.text.split("\n")
        for str in text:
            if "token"  in str:
                break
        res_json = json.loads(str)
        startTime = res_json["startTime"]
        timeModelLoad = res_json["time_model_load"]
        endTime = res_json["endTime"]
        excutionTime = res_json["runtime"]
        ip = res_json['ip']
        df = pd.DataFrame(columns=res_col)
        response_time = time.time()
        run_time = response_time - invoke_time
        df.loc[0] = [fname, invoke_time, timeModelLoad, startTime, endTime, excutionTime, response_time, run_time,ip]
        df.to_csv(cwd+'/dl.csv', mode='a', header=False, index=False)
        # return fname, invoke_time, timeModelLoad, startTime, endTime, excutionTime, response_time, run_time,ip

async def main(vol,fname):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for i in range(vol):
            tasks.append(invoke_function(req=i, session=session,fname=fname))
        results = await asyncio.gather(*tasks, return_exceptions=True)
    return results
    

In [ ]:
for i in Stream_req:
    st = time.time()
    reqs = i
    lgth = len(action_functions)
    avg_vol = int(reqs/lgth)
    for fname in action_functions:
        data = asyncio.run(main(avg_vol,fname))
        # df_res = pd.DataFrame(columns=res_col)
        # for item in data:
        #     if isinstance(item, Exception):
        #         print(item)
        #     else: 
        #         fname, invoke_time, timeModelLoad, startTime, endTime, excutionTime, response_time, run_time,ip
        #         df = pd.json_normalize(data)
        #         df.drop(['token'], axis=1, inplace=True)
        #         df['function'] = fname
        #         df['invoke_time'] = invoke_time
        #         df['startTime'] = startTime
        #         df['timeModelLoad'] = timeModelLoad
        #         df['endTime'] = endTime
        #         df['excutionTime'] = excutionTime
        #         df['response_time'] = response_time
        #         df['run_time'] = run_time
        #         df['ip'] = ip
        #         df_res = pd.concat([df_res, df], axis=0)
        # df.to_csv(cwd+'/dl.csv', mode='a', header=False, index=False)
    et = time.time()
    if et - st < 60:
        time.sleep(60-(et - st))